# Assignment 1
**Credits**: Federico Ruggeri, Eleonora Mancini, Paolo Torroni

**Keywords**: Sexism Detection, Multi-class Classification, RNNs, Transformers, Huggingface



# Contact
For any doubt, question, issue or help, you can always contact us at the following email addresses:

Teaching Assistants:

- Federico Ruggeri -> federico.ruggeri6@unibo.it
- Eleonora Mancini -> e.mancini@unibo.it

Professor:
- Paolo Torroni -> p.torroni@unibo.it

# Introduction
You are asked to address the [EXIST 2023 Task 1](https://clef2023.clef-initiative.eu/index.php?page=Pages/labs.html#EXIST) on sexism detection.

## Problem Definition
The systems have to decide whether or not a given tweet contains or describes sexist expressions or behaviors (i.e., it is sexist itself, describes a sexist situation or criticizes a sexist behavior).

### Examples:

**Text**: *Can’t go a day without women womening*

**Label**: Sexist

**Text**: *''Society's set norms! Happy men's day though!#weareequal''*

**Label**: Not sexist

# [Task 1 - 1.0 points] Corpus

We have preparared a small version of EXIST dataset in our dedicated [Github repository](https://github.com/lt-nlp-lab-unibo/nlp-course-material/tree/main/2024-2025/Assignment%201/data).

Check the `A1/data` folder. It contains 3 `.json` files representing `training`, `validation` and `test` sets.

The three sets are slightly unbalanced, with a bias toward the `Non-sexist` class.



### Dataset Description
- The dataset contains tweets in both English and Spanish.
- There are labels for multiple tasks, but we are focusing on **Task 1**.
- For Task 1, soft labels are assigned by six annotators.
- The labels for Task 1 represent whether the tweet is sexist ("YES") or not ("NO").







### Example


    "203260": {
        "id_EXIST": "203260",
        "lang": "en",
        "tweet": "ik when mandy says “you look like a whore” i look cute as FUCK",
        "number_annotators": 6,
        "annotators": ["Annotator_473", "Annotator_474", "Annotator_475", "Annotator_476", "Annotator_477", "Annotator_27"],
        "gender_annotators": ["F", "F", "M", "M", "M", "F"],
        "age_annotators": ["18-22", "23-45", "18-22", "23-45", "46+", "46+"],
        "labels_task1": ["YES", "YES", "YES", "NO", "YES", "YES"],
        "labels_task2": ["DIRECT", "DIRECT", "REPORTED", "-", "JUDGEMENTAL", "REPORTED"],
        "labels_task3": [
          ["STEREOTYPING-DOMINANCE"],
          ["OBJECTIFICATION"],
          ["SEXUAL-VIOLENCE"],
          ["-"],
          ["STEREOTYPING-DOMINANCE", "OBJECTIFICATION"],
          ["OBJECTIFICATION"]
        ],
        "split": "TRAIN_EN"
      }
    }

### Instructions
1. **Download** the `A1/data` folder.
2. **Load** the three JSON files and encode them as pandas dataframes.
3. **Generate hard labels** for Task 1 using majority voting and store them in a new dataframe column called `hard_label_task1`. Items without a clear majority will be removed from the dataset.
4. **Filter the DataFrame** to keep only rows where the `lang` column is `'en'`.
5. **Remove unwanted columns**: Keep only `id_EXIST`, `lang`, `tweet`, and `hard_label_task1`.
6. **Encode the `hard_label_task1` column**: Use 1 to represent "YES" and 0 to represent "NO".

In [1]:
pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 12.0 MB/s eta 0:00:00


In [2]:
pip install pyspellchecker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 57.3 MB/s eta 0:00:00


In [3]:
pip install textblob

### 2. Loading of the three JSON files as dataframes

In [5]:
import pandas as pd

#data_train = pd.read_json('data/training.json')
#data_train = pd.read_json('content/training.json') #Lorenzo
data_train = pd.read_json('/content/training.json') #Diego


df_train = data_train.T
df_train.reset_index(inplace=True)
df_train.rename(columns={'index': 'id'}, inplace=True)

#data_test = pd.read_json('data/test.json')
#data_test = pd.read_json('content/test.json') #Lorenzo
data_test = pd.read_json('/content/test.json') #Diego

df_test = data_test.T
df_test.reset_index(inplace=True)
df_test.rename(columns={'index': 'id'}, inplace=True)

#data_val = pd.read_json('data/validation.json')
#data_val = pd.read_json('content/validation.json') #Lorenzo
data_val = pd.read_json('/content/validation.json') #Diego


df_val = data_val.T
df_val.reset_index(inplace=True)
df_val.rename(columns={'index': 'id'}, inplace=True)

# DataFrame displaying
#print(df_train.head())

### 3. Generate hard_label_task1

In [6]:
from collections import Counter

def majority_vote(labels):
    label_counts = Counter(labels)
    most_common = label_counts.most_common(1)
    if len(most_common) > 0 and most_common[0][1] > len(labels) / 2:
        return most_common[0][0]
    return None

df_train['hard_label_task1'] = df_train['labels_task1'].apply(majority_vote)
df_train = df_train.dropna(subset=['hard_label_task1'])
df_train.reset_index(drop=True, inplace=True)

df_test['hard_label_task1'] = df_test['labels_task1'].apply(majority_vote)
df_test = df_test.dropna(subset=['hard_label_task1'])
df_test.reset_index(drop=True, inplace=True)

df_val['hard_label_task1'] = df_val['labels_task1'].apply(majority_vote)
df_val = df_val.dropna(subset=['hard_label_task1'])
df_val.reset_index(drop=True, inplace=True)


### 4. Filter DataFrame

In [7]:
df_train = df_train[df_train['lang'] == 'en']
df_test = df_test[df_test['lang'] == 'en']
df_val = df_val[df_val['lang'] == 'en']

### 5. Remove Unwanted Columns

In [8]:
df_train = df_train[['id_EXIST', 'lang', 'tweet', 'hard_label_task1']]
df_test = df_test[['id_EXIST', 'lang', 'tweet', 'hard_label_task1']]
df_val = df_val[['id_EXIST', 'lang', 'tweet', 'hard_label_task1']]

### 6. hard_label_task1 encoding

In [9]:
df_train['hard_label_task1'] = df_train['hard_label_task1'].replace({'YES': 1, 'NO': 0})
df_test['hard_label_task1'] = df_test['hard_label_task1'].replace({'YES': 1, 'NO': 0})
df_val['hard_label_task1'] = df_val['hard_label_task1'].replace({'YES': 1, 'NO': 0})

<ipython-input-9-956ecd6188b9>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_train['hard_label_task1'] = df_train['hard_label_task1'].replace({'YES': 1, 'NO': 0})
<ipython-input-9-956ecd6188b9>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_test['hard_label_task1'] = df_test['hard_label_task1'].replace({'YES': 1, 'NO': 0})
<ipython-input-9-956ecd6188b9>:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_obje

# [Task2 - 0.5 points] Data Cleaning
In the context of tweets, we have noisy and informal data that often includes unnecessary elements like emojis, hashtags, mentions, and URLs. These elements may interfere with the text analysis.



### Instructions
- **Remove emojis** from the tweets.
- **Remove hashtags** (e.g., `#example`).
- **Remove mentions** such as `@user`.
- **Remove URLs** from the tweets.
- **Remove special characters and symbols**.
- **Remove specific quote characters** (e.g., curly quotes).
- **Perform lemmatization** to reduce words to their base form.

In [10]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet

nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')

lemmatizer = WordNetLemmatizer()

def get_wordnet_key(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return 'n'

#Lemmatize each token
def lem_text(text: str):
  tokens = word_tokenize(text)
  tagged = pos_tag(tokens)
  words = [lemmatizer.lemmatize(word, get_wordnet_key(tag)) for word, tag in tagged]
  return " ".join(words)

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


In [11]:
import emoji
import re
import nltk
from tqdm import tqdm
import unicodedata
from textblob import TextBlob
#from nltk.tokenize import word_tokenize
from emoji import *
from nltk import word_tokenize


# necessary for being able to tokenize
nltk.download('punkt_tab')
nltk.download('punkt')

def correct_spelling(text):
  blob = TextBlob(text)
  corrected_text = blob.correct()
  return str(corrected_text)

def remove_style(text):
    # Normalize text into the closest ASCII equivalent
    return ''.join(
        c for c in unicodedata.normalize('NFKC', text)
        if not unicodedata.combining(c)  # Exclude combining marks
    )

def split_merge_word(text):
    # Use regex to find boundaries between lowercase and uppercase
    return re.sub(r'(?<=[a-z])(?=[A-Z])', ' ', text)

def replace_space(text):
  return re.sub(r'\s+', ' ', text).strip()

# Function to clean and preprocess tweets
def clean_tweet(tweet):
    # Remove mentions (@user)
    tweet = re.sub(r'@\w+', ' ', tweet)
    # Remove hashtags (#example)
    tweet = re.sub(r'#\w+', ' ', tweet)
    # Remove URLs
    tweet = re.sub(r'http\S+|www.\S+', ' ', tweet)
    # Remove special characters and symbols
    tweet = re.sub(r'[^\w\s]', ' ', tweet)
    # Remove emojis
    tweet = replace_emoji(tweet, ' ')
    # Remove specific quote characters (e.g., curly quotes)
    cleaned_tweet = tweet.replace('“', ' ').replace('”', ' ').replace('’', " ").replace("‘"," ").replace('"', " ").replace("'", " ")

    return cleaned_tweet

def clean_column_dataset(df_column):
  cleaned_tweets = []

  for tweet in tqdm(df_column):
    cleaned_tweet = clean_tweet(tweet)   #Clean the text
    lem_tweet = lem_text(cleaned_tweet) #Lemmatize the text
    lem_tweet_split = split_merge_word(lem_tweet) #Split words like "endYou"
    norm_tweet = remove_style(lem_tweet_split) #remove bold and italic style
    lowercase_tweet = norm_tweet.lower() #lower case the dataset
    cleaned_tweets.append(lowercase_tweet)  #Save the results
  return cleaned_tweets

df_train['cleaned_tweet'] = clean_column_dataset(df_train['tweet'])
df_val['cleaned_tweet'] = clean_column_dataset(df_val['tweet'])
df_test['cleaned_tweet'] = clean_column_dataset(df_test['tweet'])


print(df_train[['tweet', 'cleaned_tweet']].head())

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
100%|██████████| 286/286 [00:00<00:00, 377.29it/s]

                                                  tweet  \
3194  Writing a uni essay in my local pub with a cof...   
3195  @UniversalORL it is 2021 not 1921. I dont appr...   
3196  According to a customer I have plenty of time ...   
3197  So only 'blokes' drink beer? Sorry, but if you...   
3198  New to the shelves this week - looking forward...   

                                          cleaned_tweet  
3194  writing a uni essay in my local pub with a cof...  
3195  it be 2021 not 1921 i dont appreciate that on ...  
3196  according to a customer i have plenty of time ...  
3197  so only blokes drink beer sorry but if you are...  
3198  new to the shelf this week look forward to rea...  


In [12]:
df_train[['tweet', 'cleaned_tweet']].head(100)

,tweet,cleaned_tweet
3194,Writing a uni essay in my local pub with a cof...,writing a uni essay in my local pub with a cof...
3195,@UniversalORL it is 2021 not 1921. I dont appr...,it be 2021 not 1921 i dont appreciate that on ...
3196,According to a customer I have plenty of time ...,according to a customer i have plenty of time ...
3197,"So only 'blokes' drink beer? Sorry, but if you...",so only blokes drink beer sorry but if you are...
3198,New to the shelves this week - looking forward...,new to the shelf this week look forward to rea...
...,...,...
3289,They can fight for Hijabs but not against Trip...,they can fight for hijabs but not against trip...
3290,"The whiskey and cigars, the $99 dollar seminar...",the whiskey and cigars the 99 dollar seminar t...
3291,I would be glad to see the violence on Twitter...,i would be glad to see the violence on twitter...
3292,@ProteanRedux @HugoThePinkCat @DeclarationOn @...,action aid conduct a survey on street harassme...


# [Task 3 - 0.5 points] Text Encoding
To train a neural sexism classifier, you first need to encode text into numerical format.




### Instructions

* Embed words using **GloVe embeddings**.
* You are **free** to pick any embedding dimension.





In [13]:
from typing import Dict, List
from collections import OrderedDict
from tqdm import tqdm
import pandas as pd

def build_vocabulary(df: pd.DataFrame) -> (Dict[int, str], Dict[str, int], List[str]):
    """
    Given a dataset, builds the corresponding word vocabulary.

    :param df: dataset from which we want to build the word vocabulary (pandas.DataFrame)
    :return:
      - word vocabulary: vocabulary index to word
      - inverse word vocabulary: word to vocabulary index
      - word listing: set of unique terms that build up the vocabulary
    """
    idx_to_word = OrderedDict()
    word_to_idx = OrderedDict()

    curr_idx = 0
    for sentence in tqdm(df.text.values):
        tokens = sentence.split()
        for token in tokens:
            if token not in word_to_idx:
                word_to_idx[token] = curr_idx
                idx_to_word[curr_idx] = token
                curr_idx += 1

    word_to_idx["UNK"] = curr_idx
    idx_to_word[curr_idx] = 'UNK'

    word_listing = list(idx_to_word.values())
    return idx_to_word, word_to_idx, word_listing

In [14]:
df_train['text'] = df_train['cleaned_tweet']
df_test['text'] = df_test['cleaned_tweet']
df_val['text'] = df_val['cleaned_tweet']

In [15]:
idx_to_word_train, word_to_idx_train, word_listing_train = build_vocabulary(df_train)
idx_to_word_test, word_to_idx_test, word_listing_test = build_vocabulary(df_test)
idx_to_word_val, word_to_idx_val, word_listing_val = build_vocabulary(df_val)

100%|██████████| 158/158 [00:00<00:00, 47400.05it/s]


In [16]:
def evaluate_vocabulary(idx_to_word: Dict[int, str], word_to_idx: Dict[str, int],
                        word_listing: List[str], df: pd.DataFrame, check_default_size: bool = False):
    print("[Vocabulary Evaluation] Size checking...")
    assert len(idx_to_word) == len(word_to_idx)
    assert len(idx_to_word) == len(word_listing)

    print("[Vocabulary Evaluation] Content checking...")
    for i in tqdm(range(0, len(idx_to_word))):
        assert idx_to_word[i] in word_to_idx
        assert word_to_idx[idx_to_word[i]] == i

    print("[Vocabulary Evaluation] Consistency checking...")
    _, _, first_word_listing = build_vocabulary(df)
    _, _, second_word_listing = build_vocabulary(df)
    assert first_word_listing == second_word_listing

    print("[Vocabulary Evaluation] Toy example checking...")
    toy_df = pd.DataFrame.from_dict({
        'text': ["all that glitters is not gold", "all in all i like this assignment"]
    })
    _, _, toy_word_listing = build_vocabulary(toy_df)
    toy_valid_vocabulary = set(' '.join(toy_df.text.values).split())
    assert set(toy_word_listing) == toy_valid_vocabulary

In [ ]:
print("Vocabulary evaluation...")
evaluate_vocabulary(idx_to_word_train, word_to_idx_train, word_listing_train, df_train)
print("Evaluation completed!")

Vocabulary evaluation...
[Vocabulary Evaluation] Size checking...
[Vocabulary Evaluation] Content checking...


100%|██████████| 9388/9388 [00:00<00:00, 633718.93it/s]


[Vocabulary Evaluation] Consistency checking...


100%|██████████| 2870/2870 [00:00<00:00, 144812.12it/s]


[Vocabulary Evaluation] Toy example checking...


100%|██████████| 2/2 [00:00<00:00, 11554.56it/s]


AssertionError: 

In [17]:
import gensim
import gensim.downloader as gloader

def load_embedding_glove_model(embedding_dimension: int = 50) -> gensim.models.keyedvectors.KeyedVectors:
    download_path = "glove-wiki-gigaword-{}".format(embedding_dimension)
    try:
        emb_model = gloader.load(download_path)
    except ValueError as e:
        print("Invalid embedding model name! Check the embedding dimension:")
        print("Glove: 50, 100, 200, 300")
        raise e

    return emb_model

In [18]:
embedding_model = load_embedding_glove_model(embedding_dimension=50)

[==================================================] 100.0% 66.0/66.0MB downloaded


### Note : What about OOV tokens?
   * All the tokens in the **training** set that are not in GloVe **must** be added to the vocabulary.
   * For the remaining tokens (i.e., OOV in the validation and test sets), you have to assign them a **special token** (e.g., [UNK]) and a **static** embedding.
   * You are **free** to define the static embedding using any strategy (e.g., random, neighbourhood, etc...)



### More about OOV

For a given token:

* **If in train set**: add to vocabulary and assign an embedding (use GloVe if token in GloVe, custom embedding otherwise).
* **If in val/test set**: assign special token if not in vocabulary and assign custom embedding.

Your vocabulary **should**:

* Contain all tokens in train set; or
* Union of tokens in train set and in GloVe $\rightarrow$ we make use of existing knowledge!

In [19]:
def check_OOV_terms(embedding_model: gensim.models.keyedvectors.KeyedVectors,
                    word_listing: List[str]):
    """
    Checks differences between pre-trained embedding model vocabulary
    and dataset specific vocabulary in order to highlight out-of-vocabulary terms.

    :param embedding_model: pre-trained word embedding model (gensim wrapper)
    :param word_listing: dataset specific vocabulary (list)

    :return
        - list of OOV terms
    """
    embedding_vocabulary = set(embedding_model.key_to_index.keys())
    oov = set(word_listing).difference(embedding_vocabulary)
    return list(oov)

In [20]:
oov_terms_train = check_OOV_terms(embedding_model, word_listing_train)
oov_percentage_train = float(len(oov_terms_train)) * 100 / len(word_listing_train)
print(f"Total OOV terms: {len(oov_terms_train)} ({oov_percentage_train:.2f}%)")

oov_terms_test = check_OOV_terms(embedding_model, word_listing_test)
oov_percentage_test = float(len(oov_terms_test)) * 100 / len(word_listing_test)
print(f"Total OOV terms: {len(oov_terms_test)} ({oov_percentage_test:.2f}%)")

oov_terms_val = check_OOV_terms(embedding_model, word_listing_val)
oov_percentage_val = float(len(oov_terms_val)) * 100 / len(word_listing_val)
print(f"Total OOV terms: {len(oov_terms_val)} ({oov_percentage_val:.2f}%)")

Total OOV terms: 880 (9.37%)
Total OOV terms: 102 (4.72%)
Total OOV terms: 68 (4.42%)


In [ ]:
for oov in oov_terms_train:
  print(oov)

collab
5days
twtjesus
myt
motuary
likeyou
dvmb
ekyo
deadass
flyering
suguha
carousel30
godess
antifa
erens
jtrump
dforce
homosapiens
otherized
weightlift
chaturbate
wetin
wowsers
scoatlan
peacefuls
000s
trynna
aytl3
gimbap
30yo
mansplain
tinnie
porngirl
smtreading
fkn
qristianity
lanez
fatphobic
findomgoddess
transness
notallmen
paperswritingpro
talaga
来自
gooning
inmoral
wachana
38original
big0ted
journo
amrylin
country10
p5srm7ai
senary
transphobes
shiiit
oloriburuku
gcfems
dababy
wgtn
yoohan
yonke
purebloods
anyhoo
pathologises
hoseok
humberman
pandra
yrboi
adorime
rendelshem
yooooooo
loooose
imho
karke
bakubussy
bhululu
trumpanzees
hooooly
rewatching
nonary
catholism
bitvh
chefayo
fffffffuck
overfeed
yeee
brotopia
fakesswi
dvmbses
truefender
maaaan
ọnịcha
itseems
sherni
othering
quinary
65purchase
lalalala
pomlázka
wokeness
immerwahr
pppl
wh0r3s
sheisty
lxp
bowcaster
moots10
cker
ranpo
poet2021
penised
wildturtle
cishet
futanari
65k
mpeleke
vaxxers
quemix
octonary
vestors
covid2
cri

In [21]:
import numpy as np
from gensim.models import KeyedVectors

def add_oov_terms_with_batches(embedding_model: KeyedVectors, oov_terms: List[str], vector_size: int = None, batch_size: int = 1000):
    vector_size = vector_size or embedding_model.vector_size

    # Create a new KeyedVectors object
    new_kv = KeyedVectors(vector_size)

    # Prepare data for batch addition
    words = list(embedding_model.key_to_index.keys()) + oov_terms
    vectors = [embedding_model[word] for word in embedding_model.key_to_index] + [np.random.uniform(-0.1, 0.1, vector_size) for _ in oov_terms]

    # Add vectors in batches
    for i in range(0, len(words), batch_size):
        batch_words = words[i:i + batch_size]
        batch_vectors = vectors[i:i + batch_size]
        new_kv.add_vectors(batch_words, batch_vectors)

    return new_kv

# Example usage:
vector_size = embedding_model.vector_size
extended_model = add_oov_terms_with_batches(embedding_model, oov_terms_train, vector_size, batch_size=1000)

In [22]:
# Assuming you have a KeyedVectors object `embedding_model` and a list `oov_terms`
extended_model = add_oov_terms_with_batches(embedding_model, oov_terms_train, batch_size=1000)

# Verify the size of the new vocabulary
print("Extended vocabulary size:", len(extended_model.key_to_index))


Extended vocabulary size: 400880


In [23]:
import numpy as np
import gensim

def assign_static_embeddings(oov_terms, embedding_dim):
    oov_to_token = {term: f"[UNK]" for i, term in enumerate(oov_terms)}

    np.random.seed(42)  # For reproducibility
    static_embeddings = {
        token: np.random.uniform(-0.1, 0.1, embedding_dim)
        for token in oov_to_token.values()
    }

    return oov_to_token, static_embeddings

embedding_dim = embedding_model.vector_size

special_token_test, static_embedding_test = assign_static_embeddings(oov_terms_test, embedding_dim)
special_token_val, static_embedding_val = assign_static_embeddings(oov_terms_val, embedding_dim)

# [Task 4 - 1.0 points] Model definition

You are now tasked to define your sexism classifier.




### Instructions

* **Baseline**: implement a Bidirectional LSTM with a Dense layer on top.
* You are **free** to experiment with hyper-parameters to define the baseline model.

* **Model 1**: add an additional LSTM layer to the Baseline model.

In [39]:
import tensorflow as tf
import tensorflow.keras as keras
from keras.optimizers import AdamW
from keras.regularizers import l2
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import Bidirectional
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Dropout

def getBaselineModel(vocab_size, embedding_dimension, embedding_matrix, n_units = 128):
    model = Sequential([
        Embedding(input_dim=vocab_size, output_dim=embedding_dimension, weights=embedding_matrix, mask_zero=True, name='encoder_embedding_baseline'),
        Bidirectional(LSTM(n_units), return_sequences=False),
        Dense(1, activation='sigmoid', kernel_regularizer=l2(0.05)),
        #TimeDistributed(Dense(units=len(-----), activation='softmax'), name = 'timedistr_dense_layer')),
    ])

    model.compile(loss='binary_crossentropy', optimizer=AdamW(), metrics=['accuracy'])
    return model

def getBaselineModel_mod(vocab_size, embedding_dimension, embedding_matrix):
    model = Sequential([
        Embedding(input_dim=vocab_size, output_dim=embedding_dimension, weights=embedding_matrix, mask_zero=True, name='encoder_embedding_baseline'),
        Bidirectional(LSTM(128, return_sequences=False)),
        Dropout(0.5),
        Dense(1, activation='sigmoid'),
    ])

    model.compile(loss='binary_crossentropy', optimizer=AdamW(), metrics=['accuracy'])
    return model

def getModel1(vocab_size, embedding_dimension, embedding_matrix, n_units_1 = 128, n_units_2 = 64 ):
    model = Sequential([
        Embedding(input_dim=vocab_size, output_dim=embedding_dimension, weights=embedding_matrix, mask_zero=True, name='encoder_embedding_model1'),
        Bidirectional(LSTM(n_units_1, return_sequences=True)),
        Bidirectional(LSTM(n_units_2), return_sequences=False),
        Dense(1, activation='sigmoid'),
    ])
    model.compile(loss='binary_crossentropy', optimizer=AdamW(), metrics=['accuracy'])
    return model

### Token to embedding mapping

You can follow two approaches for encoding tokens in your classifier.

### Work directly with embeddings

- Compute the embedding of each input token
- Feed the mini-batches of shape (batch_size, # tokens, embedding_dim) to your model

### Work with Embedding layer

- Encode input tokens to token ids
- Define a Embedding layer as the first layer of your model
- Compute the embedding matrix of all known tokens (i.e., tokens in your vocabulary)
- Initialize the Embedding layer with the computed embedding matrix
- You are **free** to set the Embedding layer trainable or not

In [24]:
def build_embedding_matrix(embedding_model: gensim.models.keyedvectors.KeyedVectors,
                           embedding_dimension: int,
                           word_to_idx: Dict[str, int],
                           vocab_size: int,
                           oov_terms: List[str]) -> np.ndarray:
    """
    Builds the embedding matrix of a specific dataset given a pre-trained word embedding model

    :param embedding_model: pre-trained word embedding model (gensim wrapper)
    :param word_to_idx: vocabulary map (word -> index) (dict)
    :param vocab_size: size of the vocabulary
    :param oov_terms: list of OOV terms (list)

    :return
        - embedding matrix that assigns a high dimensional vector to each word in the dataset specific vocabulary (shape |V| x d)
    """
    embedding_matrix = np.zeros((vocab_size, embedding_dimension), dtype=np.float32)
    for word, idx in tqdm(word_to_idx.items()):
        if word == 'UNK':
          embedding_matrix[idx] = np.zeros(embedding_dimension)
        else:
          try:
              embedding_vector = embedding_model[word]
          except (KeyError, TypeError):
              embedding_vector = np.random.uniform(low=-0.05, high=0.05, size=embedding_dimension)

          embedding_matrix[idx] = embedding_vector

    return embedding_matrix

In [54]:
# Testing
embedding_dimension = 50
embedding_matrix = build_embedding_matrix(embedding_model, embedding_dimension, word_to_idx_train, len(word_to_idx_train), oov_terms_train)
print(f"Embedding matrix shape: {embedding_matrix.shape}")

100%|██████████| 9388/9388 [00:00<00:00, 135257.84it/s]


Embedding matrix shape: (9388, 50)


100%|██████████| 1538/1538 [00:00<00:00, 228259.42it/s]


Embedding matrix (test set) shape: (2159, 50)
Embedding matrix (validation set) shape: (1538, 50)


In [69]:
embedding = tf.keras.layers.Embedding(input_dim=len(word_to_idx_train),
                                      output_dim=50,                    #embedding dimension
                                      weights=[embedding_matrix],
                                      mask_zero=True,                   # automatically masks padding tokens
                                      name='encoder_embedding')

In [86]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

def convert_tokens_to_indices(tokenized_sentences, word_to_idx, unk_token='UNK'):
    unk_index = word_to_idx.get(unk_token, 0)  # Default to 0 if UNK is not in dictionary
    sequences = [
        [word_to_idx.get(token, unk_index) for token in sentence]
        for sentence in tokenized_sentences
    ]
    return sequences

# 1. Convert Tokenized Sentences to Indices
tokenized_sentences = df_train['cleaned_tweet'].tolist()  # Replace with your dataframe column
sequences = convert_tokens_to_indices(tokenized_sentences, word_to_idx_train)

# 2. Pad Sequences
max_sequence_length = 50  # Adjust based on your dataset or experiment with different lengths
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='post', truncating='post')

# Now padded_sequences is ready to be used as input for training
print("Shape of padded sequences:", np.shape(padded_sequences))

tokenized_sentences_val = df_val['cleaned_tweet'].tolist()  # Replace with your dataframe column
sequences_val = convert_tokens_to_indices(tokenized_sentences_val, word_to_idx_val)

# 2. Pad Sequences
max_sequence_length = 50  # Adjust based on your dataset or experiment with different lengths
padded_sequences_val = pad_sequences(sequences_val, maxlen=max_sequence_length, padding='post', truncating='post')

# Now padded_sequences is ready to be used as input for training
print("Shape of padded sequences:", np.shape(padded_sequences_val))

Shape of padded sequences: (2870, 50)
Shape of padded sequences: (158, 50)


### Padding

Pay attention to padding tokens!

Your model **should not** be penalized on those tokens.

#### How to?

There are two main ways.

However, their implementation depends on the neural library you are using.

- Embedding layer
- Custom loss to compute average cross-entropy on non-padding tokens only

**Note**: This is a **recommendation**, but we **do not penalize** for missing workarounds.

# [Task 5 - 1.0 points] Training and Evaluation

You are now tasked to train and evaluate the Baseline and Model 1.



### Instructions

* Train **all** models on the train set.
* Evaluate **all** models on the validation set.
* Compute metrics on the validation set.
* Pick **at least** three seeds for robust estimation.
* Pick the **best** performing model according to the observed validation set performance.
* Evaluate your models using macro F1-score.

In [96]:
# Assuming X_train, X_val, y_train, y_val are already prepared
X_train = padded_sequences # Sequences of token indices for training
y_train = df_train['hard_label_task1']  # Binary labels for training
X_val = padded_sequences_val  # Sequences of token indices for validation
y_val = df_val['hard_label_task1']    # Binary labels for validation

# Define hyperparameters
batch_size = 64
epochs = 10

# Get BaselineModel
vocab_size = len(word_to_idx_train)
baseline_model = getBaselineModel_mod(vocab_size, embedding_dimension, [embedding_matrix])

# Train BaselineModel
history_baseline = baseline_model.fit(
    X_train,
    y_train,
    validation_data=(X_val, y_val),
    batch_size=batch_size,
    epochs=epochs
)

# Get Model1
model1 = getModel1(vocab_size, embedding_dimension, [embedding_matrix])

# Train Model1
history_model1 = model1.fit(
    X_train,
    y_train,
    validation_data=(X_val, y_val),
    batch_size=batch_size,
    epochs=epochs
)


Epoch 1/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 7s 20ms/step - accuracy: 0.5697 - loss: 0.6890 - val_accuracy: 0.5696 - val_loss: 0.6977
Epoch 2/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6014 - loss: 0.6723 - val_accuracy: 0.5696 - val_loss: 0.7114
Epoch 3/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5858 - loss: 0.6698 - val_accuracy: 0.5696 - val_loss: 0.7245
Epoch 4/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6052 - loss: 0.6648 - val_accuracy: 0.5696 - val_loss: 0.7571
Epoch 5/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6107 - loss: 0.6575 - val_accuracy: 0.5696 - val_loss: 0.7396
Epoch 6/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5989 - loss: 0.6638 - val_accuracy: 0.5696 - val_loss: 0.7802
Epoch 7/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.6320 - loss: 0.6430 - val_accuracy: 0.5696 - val_loss: 0.8353
Epoch 8/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.6280 - loss: 0.6443 - val_accuracy: 0.5696 - val_

ValueError: Unrecognized keyword arguments passed to Bidirectional: {'return_sequences': False}

# [Task 6 - 1.0 points] Transformers

In this section, you will use a transformer model specifically trained for hate speech detection, namely [Twitter-roBERTa-base for Hate Speech Detection](https://huggingface.co/cardiffnlp/twitter-roberta-base-hate).




### Relevant Material
- Tutorial 3

### Instructions
1. **Load the Tokenizer and Model**

2. **Preprocess the Dataset**:
   You will need to preprocess your dataset to prepare it for input into the model. Tokenize your text data using the appropriate tokenizer and ensure it is formatted correctly.

   **Note**: You have to use the plain text of the dataset and not the version that you tokenized before, as you need to tokenize the cleaned text obtained after the initial cleaning process.

3. **Train the Model**:
   Use the `Trainer` to train the model on your training data.

4. **Evaluate the Model on the Test Set** using F1-macro.

# [Task 7 - 0.5 points] Error Analysis

### Instructions

After evaluating the model, perform a brief error analysis:

 - Review the results and identify common errors.

 - Summarize your findings regarding the errors and their impact on performance (e.g. but not limited to Out-of-Vocabulary (OOV) words, data imbalance, and performance differences between the custom model and the transformer...)
 - Suggest possible solutions to address the identified errors.



# [Task 8 - 0.5 points] Report

Wrap up your experiment in a short report (up to 2 pages).

### Instructions

* Use the NLP course report template.
* Summarize each task in the report following the provided template.

### Recommendations

The report is not a copy-paste of graphs, tables, and command outputs.

* Summarize classification performance in Table format.
* **Do not** report command outputs or screenshots.
* Report learning curves in Figure format.
* The error analysis section should summarize your findings.


# Submission

* **Submit** your report in PDF format.
* **Submit** your python notebook.
* Make sure your notebook is **well organized**, with no temporary code, commented sections, tests, etc...
* You can upload **model weights** in a cloud repository and report the link in the report.

# FAQ

Please check this frequently asked questions before contacting us

### Execution Order

You are **free** to address tasks in any order (if multiple orderings are available).

### Trainable Embeddings

You are **free** to define a trainable or non-trainable Embedding layer to load the GloVe embeddings.

### Model architecture

You **should not** change the architecture of a model (i.e., its layers).
However, you are **free** to play with their hyper-parameters.


### Neural Libraries

You are **free** to use any library of your choice to implement the networks (e.g., Keras, Tensorflow, PyTorch, JAX, etc...)

### Keras TimeDistributed Dense layer

If you are using Keras, we recommend wrapping the final Dense layer with `TimeDistributed`.

### Robust Evaluation

Each model is trained with at least 3 random seeds.

Task 4 requires you to compute the average performance over the 3 seeds and its corresponding standard deviation.

### Model Selection for Analysis

To carry out the error analysis you are **free** to either

* Pick examples or perform comparisons with an individual seed run model (e.g., Baseline seed 1337)
* Perform ensembling via, for instance, majority voting to obtain a single model.

### Error Analysis

Some topics for discussion include:
   * Precision/Recall curves.
   * Confusion matrices.
   * Specific misclassified samples.

### Bonus Points
Bonus points are arbitrarily assigned based on significant contributions such as:
- Outstanding error analysis
- Masterclass code organization
- Suitable extensions
Note that bonus points are only assigned if all task points are attributed (i.e., 6/6).

**Possible Extensions/Explorations for Bonus Points:**
- **Try other preprocessing strategies**: e.g., but not limited to, explore techniques tailored specifically for tweets or  methods that are common in social media text.
- **Experiment with other custom architectures or models from HuggingFace**
- **Explore Spanish tweets**: e.g., but not limited to, leverage multilingual models to process Spanish tweets and assess their performance compared to monolingual models.







# The End